This code come from 
https://medium.com/analytics-vidhya/building-a-simple-chatbot-in-python-using-nltk-7c8c8215ac6e


In [15]:
pip install googletrans

Nous installons la bibliothèque googleTans qui gère la traduction d'une langue à l'autre. Sa particulatité est qu'elle peut détecter une langue ce qui nous permet de donner la langue que l'on veut traduire directement

In [16]:
import nltk 
import numpy as np 
import random 
import string # pour traiter les chaînes python standard
import warnings

Nous lisons le fichier coronavirus.txt et convertirons l'ensemble du chatbot en une liste de phrases et une liste de mots pour un prétraitement ultérieur.


In [17]:
# Connecting Google Drive Data Files to Google Colab Notebook.
# Load the Google Drive helper and mount the drive.
from google.colab import drive

# This will prompt for authorization. Click the link and provide the required information.

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
f=open('/content/drive/My Drive/Coronavirus.txt','r',errors = 'ignore')
raw=f.read()
raw=raw.lower()# converts to lowercase
nltk.download('punkt') # first-time use only
nltk.download('wordnet') # first-time use only
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Nous allons maintenant définir une fonction appelée LemTokens qui prendra en entrée les jetons et retournera les jetons normalisés.

In [19]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

Ensuite, nous allons définir une fonction pour un message d'accueil par le bot, c'est-à-dire que si l'entrée d'un utilisateur est un message d'accueil, le bot doit renvoyer une réponse de salutation.ELIZA utilise un simple mot clé correspondant pour les salutations. Nous utiliserons ici le même concept.

In [20]:
GREETING_INPUTS = ("Bonjour", "Salut", "salutations", "Super", "quoi de neuf","hey",)
GREETING_RESPONSES = ["Salut", "Hey", "*hoche la tête*", "Salut", "Bonjour", "Je suis content! Vous me parlez"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

Pour générer une réponse de notre bot aux questions d'entrée, le concept de similitude des documents sera utilisé. Nous commençons donc par importer les modules nécessaires.

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

Il sera utilisé pour trouver la similitude entre les mots saisis par l'utilisateur et les mots du chatbot. Il s'agit de l'implémentation la plus simple possible d'un chatbot.
Nous définissons une réponse de fonction qui recherche dans l'énoncé de l'utilisateur un ou plusieurs mots clés connus et renvoie l'une des réponses possibles. S'il ne trouve pas l'entrée correspondant à l'un des mots clés, il renvoie une réponse: «Je suis désolé! Je ne te comprends pas »

In [23]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+" Je suis désolé! Je ne comprends pas votre question"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

Enfin, nous alimenterons les lignes que nous voulons que notre bot dise tout en commençant et en terminant une conversation en fonction de l'entrée de l'utilisateur.

In [24]:
warnings.filterwarnings('ignore')
from googletrans import Translator
translator = Translator() 
traduction_fr = ""
flag=True
print("ROBO: Je m'appelle Robo de la Team4. Je répondrai à vos questions sur le coronavirus. Si vous souhaitez quitter, tapez Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='merci' ):
            flag=False
            print("ROBO: Je vous en pris..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                traduction_fr = translator.translate(response(user_response),dest='fr')
                print(f'{traduction_fr.text}')
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Au revoir! prenez soins de vous..")

ROBO: Je m'appelle Robo de la Team4. Je répondrai à vos questions sur le coronavirus. Si vous souhaitez quitter, tapez Bye!
Bonjour
ROBO: Je suis désolé! Je ne comprends pas votre question
coronavirus
ROBO: [61] [62]

genre: alphacoronavirus; [58] espèce type: alphacoronavirus 1 (tgev)
espèces: alphacoronavirus 1, coronavirus humain 229e, coronavirus humain nl63, miniopterus bat coronavirus 1, miniopterus bat coronavirus hku8, porcine epidemic epidearirus virus, rhinolophus bat coronavirus hku2, scotophilus bat coronavirus 512
genre betacoronavirus; [59] espèces de type: coronavirus murin (mhv)
espèces: bétacoronavirus 1 (coronavirus bovin, coronavirus humain oc43), coronavirus hérisson 1, coronavirus humain hku1, coronavirus lié au syndrome respiratoire du Moyen-Orient, coronavirus murin, pipistrellus bat coronavirus hku5, rousettus bat coronavirus hku9, syndrome respiratoire aigu sévère sars-cov, sars-cov-2), tylonycteris bat coronavirus hku4
genre gammacoronavirus; [16] espèce type: